# Sourcing fire data
This notebook stitches together downloaded jsons from fire.ca.gov to create several dataframes used in the main notebook.

The underlying jsons are not provided, but one can download it for themselves from ca.fire.gov and run this notebook to create updated files.

Creating a dataframe hosting the relationship between each solar data station to the wildfire locations and a dataframe with fire lat/lons

Creating a dataframe holding the key fire data